In [1]:
import molsysmt as msm
import numpy as np
from molsysmt import pyunitwizard as puw

In [2]:
import openmm as mm
from openmm import unit
from openmm import app
from tqdm import tqdm
import numpy as np
from matplotlib import pyplot as plt

In [3]:
from parmed.openmm.reporters import EnergyMinimizerReporter

In [4]:
molsys_AF = msm.convert('1L2Y')

In [5]:
molsys_AF = msm.build.add_missing_hydrogens(molsys_AF, pH=7.4)

In [6]:
topology = msm.convert(molsys_AF, to_form='openmm.Topology')
positions = msm.get(molsys_AF, element='atom', structure_indices=0, coordinates=True)
positions = msm.pyunitwizard.convert(positions[0], to_form='openmm.unit')

In [7]:
forcefield = app.ForceField("amber14-all.xml")

In [8]:
system = forcefield.createSystem(topology, nonbondedMethod=app.NoCutoff, constraints=app.HBonds)

In [9]:
temperature = 0*unit.kelvin

integration_timestep = 2.0*unit.femtoseconds
saving_interval = 1.00*unit.picoseconds
logging_interval = 100.00*unit.picoseconds
simulation_time = 500.*unit.picoseconds

saving_steps = int(saving_interval/integration_timestep)
logging_steps = int(logging_interval/integration_timestep)
md_steps = int(simulation_time/integration_timestep)

In [10]:
friction   = 1.0/unit.picoseconds
integrator = mm.LangevinIntegrator(temperature, friction, integration_timestep)

In [11]:
platform = mm.Platform.getPlatformByName('CPU')

In [12]:
simulation = app.Simulation(topology, system, integrator, platform)
simulation.context.setPositions(positions)

In [13]:
reporter = msm.thirds.openmm.reporters.StructuresDictReporter(1, coordinates=True, potentialEnergy=True)

In [14]:
reporter = EnergyMinimizerReporter(open('nada.txt','w'))

In [ ]:
#reporter = msm.thirds.openmm.reporters.StructuresDictReporter(100, coordinates=True, potentialEnergy=True)

In [ ]:
#simulation.reporters.append(reporter)

In [15]:
simulation.minimizeEnergy(reporter=reporter)

TypeError: Wrong number or type of arguments for overloaded function 'LocalEnergyMinimizer_minimize'.
  Possible C/C++ prototypes are:
    OpenMM::LocalEnergyMinimizer::minimize(OpenMM::Context &,double,int,OpenMM::MinimizationReporter *)
    OpenMM::LocalEnergyMinimizer::minimize(OpenMM::Context &,double,int)
    OpenMM::LocalEnergyMinimizer::minimize(OpenMM::Context &,double)
    OpenMM::LocalEnergyMinimizer::minimize(OpenMM::Context &)


In [ ]:
reporter.finalize()

In [ ]:
reporter._out.close()

In [ ]:
traj_dict=reporter.finalize()

In [ ]:
state=simulation.context.getState(getEnergy=True)
state.getPotentialEnergy()

In [ ]:
msm.view(simulation)